In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

cnn = Sequential([Conv2D(filters=100, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Conv2D(filters=100, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Flatten(),
                    Dropout(0.5),
                    Dense(50),
                    Dense(35),
                    Dense(2)])

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_directory = "training"
testing_directory = "test"
training_data_generator = ImageDataGenerator()
testing_data_generator = ImageDataGenerator()
training_generator = training_data_generator.flow_from_directory(training_directory, batch_size=10, target_size=(150, 150))
testing_generator = testing_data_generator.flow_from_directory(testing_directory, batch_size=10, target_size=(150, 150))

In [ ]:
epochs = cnn.fit(training_generator, epochs=1, validation_data=testing_generator)

In [ ]:
import cv2
import numpy as np

labels_dict={0:'No mask',1:'Mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
imgsize = 4 #set image resize
camera = cv2.VideoCapture(0) #use camera
#identify front face
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
while True:
    (rval, im) = camera.read()
    im=cv2.flip(im,1,1) #mirrow the image
    imgs = cv2.resize(im, (im.shape[1] // imgsize, im.shape[0] // imgsize))
    face_rec = classifier.detectMultiScale(imgs)#detect faces multi scale
    for i in face_rec:#extract face rectgular
        (x, y, l, w) = [v * imgsize for v in i] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+w, x:x+l]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=cnn.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+l,y+w),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+l,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # stop loop by ESC
    if key == 27: #The Esc key
        break
webcam.release()
cv2.destroyAllWindows()